In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

### Preparación de datasets

In [2]:
from sklearn.preprocessing import LabelEncoder

def dataset_prep(window=3, neutral=True):
    
    train = pd.read_csv('data/trainset.csv', index_col=0)
    test  = pd.read_csv('data/testset.csv', index_col=0)
    
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    
    for col in cols1:
        train[col+'w'] = train[col].rolling(window=window).sum()
        test[col+'w'] = test[col].rolling(window=window).sum()
    
    train['exc_ret'] = train['exc_ret'].shift(periods=-1)
    test['exc_ret']  = test['exc_ret'].shift(periods=-1)
    
    train.dropna(axis=0, inplace=True)
    test.dropna(axis=0, inplace=True)
    
    cols2 = [x+'w' for x in cols1]
    
    X = train[cols2]
    y = train.exc_ret
    Xt = test[cols2]
    yt = test.exc_ret
    
    le = LabelEncoder()
    y  = le.fit_transform(y)
    yt = le.fit_transform(yt)
    
    return X, Xt, y, yt    

### Entrenamiento de Clasificadores

In [3]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_roc

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier 

In [4]:
def train_classifier(X, Xt, y, yt):
    
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression', 'Catboost']
   
    model_init = [RandomForestClassifier(),
                  XGBClassifier(),
                  lgb.LGBMClassifier(),
                  LogisticRegression(),
                  CatBoostClassifier()]
    params1 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 25),     # Maximum tree depth for base learners.
    }
    
    params2 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 25),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    params4 = {}
    
    model_params = [params1, params2, params2, params3, params4]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    for name, mod, params in zip(model_name, model_init, model_params):
        model = mod
        kf    = KFold(n_splits=5, shuffle=True, random_state=RS)
        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=kf,
                                   scoring='roc_auc', n_iter=50, verbose=1, n_jobs=-1)
        rgrid.fit(X, y)
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        ppred = rgrid.predict_proba(Xt)
        score = roc_auc_score(y_score=ppred[:, 1], y_true=yt)
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results #.sort_values(by='Test Score', ascending=False)

In [5]:
import itertools
windows = np.arange(1, 6)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = dataset_prep(window=win, neutral=neu)
    
    clfs = train_classifier(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : clfs['Model'],
     'Window'     : list(str(win)*clfs.shape[0]),
     'Neutral'    : list(str(neu)*clfs.shape[0]),
     'Train Score': clfs['Train Score'],
     'Test Score' : clfs['Test Score'],
     'Params'     : clfs['Params'],
     'Estimator'  : clfs['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    8.5s finished
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.5s finished
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


...

173:	learn: 0.6172535	total: 1.16s	remaining: 5.51s
174:	learn: 0.6171102	total: 1.18s	remaining: 5.55s
175:	learn: 0.6167000	total: 1.18s	remaining: 5.54s
176:	learn: 0.6163404	total: 1.2s	remaining: 5.57s
177:	learn: 0.6161239	total: 1.2s	remaining: 5.56s
178:	learn: 0.6158848	total: 1.21s	remaining: 5.55s
179:	learn: 0.6154726	total: 1.22s	remaining: 5.54s
180:	learn: 0.6148511	total: 1.22s	remaining: 5.53s
181:	learn: 0.6145278	total: 1.23s	remaining: 5.53s
182:	learn: 0.6136689	total: 1.24s	remaining: 5.51s
183:	learn: 0.6133531	total: 1.24s	remaining: 5.5s
184:	learn: 0.6128377	total: 1.25s	remaining: 5.5s
185:	learn: 0.6125903	total: 1.25s	remaining: 5.49s
186:	learn: 0.6123465	total: 1.26s	remaining: 5.49s
187:	learn: 0.6121678	total: 1.27s	remaining: 5.48s
188:	learn: 0.6117228	total: 1.27s	remaining: 5.47s
189:	learn: 0.6113494	total: 1.28s	remaining: 5.46s
190:	learn: 0.6107697	total: 1.29s	remaining: 5.45s
191:	learn: 0.6104369	total: 1.29s	remaining: 5.45s
192:	learn: 0.61

359:	learn: 0.5750632	total: 2.34s	remaining: 4.16s
360:	learn: 0.5747062	total: 2.35s	remaining: 4.16s
361:	learn: 0.5745816	total: 2.35s	remaining: 4.15s
362:	learn: 0.5743332	total: 2.36s	remaining: 4.14s
363:	learn: 0.5742573	total: 2.36s	remaining: 4.13s
364:	learn: 0.5737479	total: 2.37s	remaining: 4.12s
365:	learn: 0.5735431	total: 2.38s	remaining: 4.12s
366:	learn: 0.5733641	total: 2.38s	remaining: 4.11s
367:	learn: 0.5732463	total: 2.39s	remaining: 4.1s
368:	learn: 0.5729707	total: 2.39s	remaining: 4.09s
369:	learn: 0.5727029	total: 2.4s	remaining: 4.08s
370:	learn: 0.5723683	total: 2.4s	remaining: 4.08s
371:	learn: 0.5722207	total: 2.41s	remaining: 4.07s
372:	learn: 0.5720513	total: 2.42s	remaining: 4.06s
373:	learn: 0.5718184	total: 2.42s	remaining: 4.05s
374:	learn: 0.5716930	total: 2.43s	remaining: 4.05s
375:	learn: 0.5715803	total: 2.43s	remaining: 4.04s
376:	learn: 0.5714402	total: 2.44s	remaining: 4.03s
377:	learn: 0.5713196	total: 2.44s	remaining: 4.02s
378:	learn: 0.5

529:	learn: 0.5496960	total: 3.32s	remaining: 2.94s
530:	learn: 0.5496958	total: 3.32s	remaining: 2.94s
531:	learn: 0.5496528	total: 3.33s	remaining: 2.93s
532:	learn: 0.5495529	total: 3.33s	remaining: 2.92s
533:	learn: 0.5493291	total: 3.34s	remaining: 2.92s
534:	learn: 0.5493135	total: 3.35s	remaining: 2.91s
535:	learn: 0.5492649	total: 3.35s	remaining: 2.9s
536:	learn: 0.5491191	total: 3.36s	remaining: 2.89s
537:	learn: 0.5489944	total: 3.36s	remaining: 2.89s
538:	learn: 0.5489036	total: 3.37s	remaining: 2.88s
539:	learn: 0.5486813	total: 3.37s	remaining: 2.87s
540:	learn: 0.5485568	total: 3.38s	remaining: 2.87s
541:	learn: 0.5480296	total: 3.38s	remaining: 2.86s
542:	learn: 0.5479439	total: 3.39s	remaining: 2.85s
543:	learn: 0.5476652	total: 3.4s	remaining: 2.85s
544:	learn: 0.5476168	total: 3.41s	remaining: 2.84s
545:	learn: 0.5472915	total: 3.41s	remaining: 2.84s
546:	learn: 0.5469981	total: 3.42s	remaining: 2.83s
547:	learn: 0.5469450	total: 3.42s	remaining: 2.82s
548:	learn: 0.

698:	learn: 0.5304566	total: 4.28s	remaining: 1.84s
699:	learn: 0.5304552	total: 4.29s	remaining: 1.84s
700:	learn: 0.5304406	total: 4.29s	remaining: 1.83s
701:	learn: 0.5303765	total: 4.3s	remaining: 1.82s
702:	learn: 0.5300904	total: 4.3s	remaining: 1.82s
703:	learn: 0.5298737	total: 4.31s	remaining: 1.81s
704:	learn: 0.5298245	total: 4.32s	remaining: 1.81s
705:	learn: 0.5298225	total: 4.32s	remaining: 1.8s
706:	learn: 0.5295439	total: 4.33s	remaining: 1.79s
707:	learn: 0.5295414	total: 4.33s	remaining: 1.79s
708:	learn: 0.5293777	total: 4.34s	remaining: 1.78s
709:	learn: 0.5293256	total: 4.34s	remaining: 1.77s
710:	learn: 0.5292555	total: 4.35s	remaining: 1.77s
711:	learn: 0.5290759	total: 4.35s	remaining: 1.76s
712:	learn: 0.5287659	total: 4.36s	remaining: 1.75s
713:	learn: 0.5287549	total: 4.37s	remaining: 1.75s
714:	learn: 0.5287300	total: 4.37s	remaining: 1.74s
715:	learn: 0.5285408	total: 4.38s	remaining: 1.74s
716:	learn: 0.5284681	total: 4.38s	remaining: 1.73s
717:	learn: 0.5

872:	learn: 0.5125521	total: 5.25s	remaining: 763ms
873:	learn: 0.5123654	total: 5.25s	remaining: 757ms
874:	learn: 0.5122313	total: 5.26s	remaining: 751ms
875:	learn: 0.5121896	total: 5.26s	remaining: 745ms
876:	learn: 0.5121451	total: 5.27s	remaining: 739ms
877:	learn: 0.5121199	total: 5.28s	remaining: 733ms
878:	learn: 0.5119282	total: 5.28s	remaining: 727ms
879:	learn: 0.5117286	total: 5.29s	remaining: 721ms
880:	learn: 0.5117178	total: 5.29s	remaining: 715ms
881:	learn: 0.5117027	total: 5.3s	remaining: 709ms
882:	learn: 0.5116124	total: 5.3s	remaining: 703ms
883:	learn: 0.5116096	total: 5.31s	remaining: 697ms
884:	learn: 0.5115940	total: 5.32s	remaining: 691ms
885:	learn: 0.5114551	total: 5.32s	remaining: 685ms
886:	learn: 0.5114516	total: 5.33s	remaining: 679ms
887:	learn: 0.5114354	total: 5.33s	remaining: 672ms
888:	learn: 0.5113075	total: 5.34s	remaining: 666ms
889:	learn: 0.5112193	total: 5.34s	remaining: 660ms
890:	learn: 0.5112191	total: 5.35s	remaining: 654ms
891:	learn: 0.

In [7]:
resultado.sort_values(by='Train Score', ascending=False).iloc[:, :-1].to_csv('class_tblob_tw+reddit.csv')